<a href="https://colab.research.google.com/github/abyaadrafid/Deep-Reinforcement-Learning/blob/master/Vanilla_Policy_Gradient_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install swig cmake libopenmpi-dev zlib1g-dev
!pip install stable-baselines[mpi]==2.10.0 box2d box2d-kengz

Reading package lists... Done
Building dependency tree       
Reading state information... Done
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev set to manually installed.
libopenmpi-dev is already the newest version (2.1.1-8).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 35 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import gym
import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use('seaborn')

In [3]:
env = gym.make('LunarLander-v2')
env.seed(0)
print(env.action_space)
print(env.observation_space)

Discrete(4)
Box(8,)


In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [6]:
class PolicyNetwork(nn.Module):
  def __init__(self, state_size, fc1_size, fc2_size, action_size):
    super(PolicyNetwork, self).__init__()
    
    self.fc1 = nn.Linear(state_size, fc1_size)
    self.fc2 = nn.Linear(fc1_size, fc2_size)
    self.fc3 = nn.Linear(fc2_size, action_size)

  def forward(self, observation):
    state = torch.Tensor(observation).to(device)
    x = F.relu(self.fc1(state))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)

    return x

In [9]:
class PolicyGradientAgent():
  def __init__(self, lr, state_size, action_size, fc1_size = 128, fc2_size = 256,):
    self.policy_network = PolicyNetwork(state_size, fc1_size, fc2_size, action_size).to(device)
    self.optimizer = optim.Adam(self.policy_network.parameters(), lr)
    self.action_memory = []
    self.reward_memory = []

  def choose_action(self, observation):
    probabilites = F.softmax(self.policy_network.forward(observation))
    action_probs = torch.distributions.Categorical(probabilites)
    action = action_probs.sample()
    log_probs = action_probs.log_prob(action)
    self.action_memory.append(log_probs)

    return action.item()

  def store_rewards(self, reward):
    self.reward_memory.append(reward)

  def 